In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from reproject import reproject_interp
from astropy import stats
from astropy.io import fits
from astropy.table import join
from glob import glob 
import sys
import os

sys.path.append('./../')
from modules import dendro_dendro, dendro_misc, dendro_props, dendro_mask

In [2]:
# Define names and filenames...

galaxy = 'ngc1433'
galaxy_hst = galaxy
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha_sic.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name

cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/dendro' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)

rerun_all = False
rerun_masking = False

regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/Nebulae_catalogue_v3/Nebulae_catalogue_v3.fits' %root_dir

print(hstha_file)
print(muscat_file)
print(musha_file)
print(cutout_dir)
print(dendro_dir)
print(cutouts_hdus_dir)
print(regions_file)
print(regions_pickel_file)
print(sample_table_file)
print(muscat_table_file)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/muse/NGC1433_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/muse/NGC1433-0.91asec_MAPS.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/cutouts
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/dendro
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/cutouts_hdus
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/cutouts/sample.reg
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/cutouts/sample.pickel
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/Nebulae_catalogue_v3/N

In [3]:
# Loading files...
hstha_hdu = fits.open(hstha_file)[0]
muscat_hdu = fits.open(muscat_file)[0]

hstha_hdu = dendro_misc.convert_to_float32(hstha_hdu)
muscat_hdu = dendro_misc.convert_to_float32(muscat_hdu)

# Update arrays
muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
muscat_hdu.data[muscat_hdu.data==-1] = np.nan

# Interpolate masks
muscat_data_re, _ = reproject_interp(muscat_hdu, hstha_hdu.header)
muscat_data_mask = ~np.isnan(muscat_data_re)
data_outmask = hstha_hdu.data[~muscat_data_mask]

# Get RMS for whole map... 
std = stats.mad_std(data_outmask, ignore_nan=True)  # Get noise
std = stats.mad_std(data_outmask[data_outmask<20*std], ignore_nan=True)  # Get noise below threshold

# Load regions, sample table and HDUs... 
hdus_cutouts = dendro_misc.load_pickle('%s/hdus_all.pickel' %cutout_dir)
regions = dendro_misc.load_pickle(regions_pickel_file)

sample_table = dendro_misc.get_galaxyprops(galaxy, sample_table_file)
muscat_table = dendro_misc.get_museprops(galaxy, muscat_table_file)

# Load cutout hdus with smoothed, masked, and non-masked data...
hdus_file = '%s/hdus_all_withmasked.pickel' %cutout_dir
muscat_regionIDs_file =  '%s/muscat_regionIDs.pickel' %cutout_dir

if os.path.exists(hdus_file) & ~rerun_masking:
    muscat_regionIDs = muscat_table['region_ID']
    hdus = dendro_misc.load_pickle(hdus_file)
else: 
    muscat_regionIDs = muscat_table['region_ID']
    hdus = dendro_dendro.get_maskedhdus(hdus_cutouts, regions, muscat_regionIDs)
    dendro_misc.save_pickle(hdus, hdus_file)

# Get dendrogram for all... 
# props_all, hdus = dendro_dendro.get_dedro_all(hdus, sample_table, muscat_regionIDs)
props_all, hdus = dendro_dendro.get_dedro_all(hdus, sample_table, muscat_regionIDs, min_npix=9, min_value_sig=2, min_delta_sig=2, std=std)

# Add additional infomation from MUSE and save... 
props_all_muscat_table = join(props_all, muscat_table, keys='region_ID')
props_all_muscat_table['flux_corr'] = dendro_props.correct_ha_flux(props_all_muscat_table)
props_all_muscat_table['ha_lum_hst'] = dendro_props.calculate_luminosity(props_all_muscat_table['flux_corr']*1e-20, sample_table['dist'].quantity[0])   
props_all_muscat_table['region_circ_rad_pc'] = dendro_props.calculate_radius(props_all_muscat_table['region_circ_rad'], sample_table['dist'].quantity[0])  
props_all_muscat_table.rename_column('flux', 'HA6562_FLUX_HST')
props_all_muscat_table.rename_column('flux_corr', 'HA6562_FLUX_HST_CORR')
props_all_muscat_table.rename_column('ha_lum_hst', 'HA6562_LUMINOSITY_HST')

dendro_misc.save_pickle(hdus, '%s/hdus_all_withmasked_withdendro.pickel' %dendro_dir)
dendro_misc.save_pickle(props_all_muscat_table, '%s/props_all.pickel' %dendro_dir)        
props_all_muscat_table.write('%s/props_all.fits' %dendro_dir, overwrite=True)

# Save HDUs as HDU lists for each region with RegionID
dendro_dendro.get_hdulists(hdus, muscat_regionIDs, outputdir=cutouts_hdus_dir)

# Save region file of masks 
dendro_mask.get_ds9regions_circ_decor(props_all, '%s/%s_trunk_regions.reg' %(dendro_dir, galaxy))
dendro_mask.get_ds9regions_ellip_decor(props_all, '%s/%s_sigma_regions.reg' %(dendro_dir, galaxy))

# Save the mask for whole mosaic
hstha_hdu = fits.open(hstha_file)[0]
dendro_mask.get_hdumask(hstha_hdu, hdus['indexmap_trunk_hdu'], '%s/%s_trunk_mask.fits'  %(dendro_dir, galaxy))
dendro_mask.get_hdumask(hstha_hdu, hdus['indexmap_trunk_close_hdu'], '%s/%s_trunk_close_mask.fits'  %(dendro_dir, galaxy))

# Save flags for when regions are touching
hdu_mask = fits.open('%s/%s_trunk_mask.fits'  %(dendro_dir, galaxy))[0]
props_all_muscat_table = dendro_dendro.get_flag_touch(props_all_muscat_table, hdu_mask)
props_all_muscat_table.write('%s/props_all.fits' %dendro_dir, overwrite=True)

[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/cutouts/hdus_all.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/cutouts/sample.pickel
[INFO] [get_galaxyprops] Getting sample table properties for ngc1433...
[INFO] [get_MuseProps] Getting MUSE catalouge properties for ngc1433...
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/cutouts/hdus_all_withmasked.pickel


Dendrogram:   0%|          | 0/1736 [00:00<?, ?it/s]

[INFO] [get_dedro] [regionID=1.0] No dendro found...
[INFO] [get_dedro] Running dummy dendro...
[INFO] [get_dedro] [regionID=2.0] No dendro found...
[INFO] [get_dedro] [regionID=3.0] No dendro found...
[INFO] [get_dedro] [regionID=4.0] No dendro found...
[INFO] [get_dedro] [regionID=8.0] No dendro found...
[INFO] [get_dedro] [regionID=12.0] No dendro found...
[INFO] [get_dedro] [regionID=13.0] No dendro found...
[INFO] [get_dedro] [regionID=19.0] No dendro found...
[INFO] [get_dedro] [regionID=21.0] No dendro found...
[INFO] [get_dedro] [regionID=22.0] No dendro found...
[INFO] [get_dedro] [regionID=25.0] No dendro found...
[INFO] [get_dedro] [regionID=26.0] No dendro found...
[INFO] [get_dedro] [regionID=29.0] No dendro found...
[INFO] [get_dedro] [regionID=30.0] No dendro found...
[INFO] [get_dedro] [regionID=32.0] No dendro found...
[INFO] [get_dedro] [regionID=35.0] No dendro found...
[INFO] [get_dedro] [regionID=36.0] No dendro found...
[INFO] [get_dedro] [regionID=37.0] No dendro

  0%|          | 0/1736 [00:00<?, ?it/s]

[INFO] [get_ds9regions_circ_decor] Saved regions to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/dendro/ngc1433_trunk_regions.reg
[INFO] [get_ds9regions_ellip_decor] Saved regions to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/dendro/ngc1433_sigma_regions.reg


Masking regions:   0%|          | 0/1736 [00:00<?, ?it/s]

Masking regions:   0%|          | 0/1736 [00:00<?, ?it/s]

  0%|          | 0/1736 [00:00<?, ?it/s]